In [1]:
import pandas as pd

In [2]:
# read CREEDS disease signature data

data = pd.read_csv('../data/disease_signatures-v1.0.csv',
                  sep=',', header=0, index_col=0)
data.head(50)

,cell_type,ctrl_ids,curator,disease_name,do_id,geo_id,organism,pert_ids,platform,umls_cui,version
id,,,,,,,,,,,
dz:328,Muscle - Striated (Skeletal) (MMHCC),GSM4372|GSM4373|GSM4374|GSM4375|GSM4376,Joel.Dudley,Duchenne muscular dystrophy,DOID:11723,GSE466,mouse,GSM4377|GSM4378|GSM4379|GSM4380|GSM4381,GPL81,C0013264,1.0
dz:325,Ganglioneuroblastoma,GSM282582|GSM282583|GSM282587|GSM282588|GSM282589,Joel.Dudley,Nicotine addiction,NaN,GSE11208,human,GSM282584|GSM282585|GSM282586|GSM282590|GSM282...,GPL570,C0028043,1.0
dz:324,Nose,GSM286649|GSM286650|GSM286651|GSM286655|GSM286...,Joel.Dudley,Rhinovirus infection,NaN,GSE11348,human,GSM286646|GSM286647|GSM286648|GSM286652|GSM286...,GPL570,C0276447,1.0
dz:323,Myocardial tissue,GSM82393|GSM82394|GSM82395|GSM82396|GSM82397|G...,Joel.Dudley,dilated cardiomyopathy,DOID:12930,GSE3586,human,GSM82408|GSM82409|GSM82410|GSM82411|GSM82412|G...,GPL3050,C0007193,1.0
dz:322,Peripheral blood mononuclear cell,GSM66671|GSM66672|GSM66673|GSM66674|GSM66675|G...,Joel.Dudley,bacterial infectious disease,DOID:104,GSE3026,human,GSM66617|GSM66618|GSM66619|GSM66620|GSM66621|G...,GPL8300,C0004623,1.0
dz:321,Lung Tissue,GSM533882|GSM533883|GSM533884|GSM533885|GSM533...,Joel.Dudley,idiopathic interstitial pneumonia,DOID:2797,GSE21369,human,GSM533888|GSM533889|GSM533890|GSM533891|GSM533...,GPL570,C0085786,1.0
dz:320,Renal Tissue,GSM9920|GSM9921|GSM9922|GSM9923|GSM9924|GSM9925,Joel.Dudley,type 2 diabetes mellitus,DOID:9352,GSE642,mouse,GSM9926|GSM9927|GSM9928|GSM9929|GSM9930|GSM9931,GPL81,C0011860,1.0
dz:49,B Cell Lymphocyte,GSM45026|GSM45027|GSM45046|GSM45047,Joel.Dudley,leukemia,DOID:1240,GSE2399,human,GSM45036|GSM45038|GSM45042|GSM45043|GSM45056|G...,GPL1899,C0023418,1.0
dz:822,Back Skin (K5-TGFbeta1 psoriasis model),GSM684707|GSM684708|GSM684709,cadimo,psoriasis,DOID:8893,GSE27628,mouse,GSM684704|GSM684705|GSM684706,GPL1261,C0033860,1.0


In [3]:
# read into CREEDS disease signatures GMT file by lines

fin = open('../data/disease_signatures-v1.0.gmt')

flines = fin.readlines()

fin.close()

In [12]:
#process the data from the GMT file
#for each line seperate disease names, disease ID, genes, and direction of regulation up or down (up & dn respectively)

for i in range(len(flines)):
    actual_disease = flines[i][:-1].split('\t')
    diseaseId = actual_disease[1]
    direction = actual_disease[0].split('-')[1]
    genes = actual_disease[2:] 


In [13]:
#creating a dictionary for genes, disease ID and direction

diseses_genes = {}
diseses_genes[diseaseId] = {} 
diseses_genes[diseaseId][direction] = genes

In [14]:
#process the data from the GMT file
#for each line seperate disease names, disease ID, genes, and direction of regulation up or down (up & dn respectively)
#add only human genes to the dictionary 

diseses_genes = {}

for i in range(len(flines)):
    actual_disese = flines[i][:-1].split('\t')
    diseaseId = actual_disese[1]
    direction = actual_disese[0].split('-')[-1]
    genes = actual_disese[2:]
    genes = [x.upper() for x in genes]
        
   
    if diseaseId not in diseses_genes.keys():
        diseses_genes[diseaseId] = {} 
    diseses_genes[diseaseId][direction] = genes


### calculating the simialrity between genes regarding the direction of the regulation (up and down)

In [15]:
#create a function to calculate Jaccard index 

def calculate_jacard(geneset1, geneset2):
    intersection = geneset1 & geneset2
    union = geneset1 | geneset2
    return len(intersection) / len(union)

In [16]:
#creating 4 dataframes for the similarity (jaccard values)
#each dataframe for a combination of up and down regulated genes

similarity_up_up = pd.DataFrame(index=diseses_genes.keys(), columns=diseses_genes.keys())
similarity_dn_dn = pd.DataFrame(index=diseses_genes.keys(), columns=diseses_genes.keys())
similarity_up_dn = pd.DataFrame(index=diseses_genes.keys(), columns=diseses_genes.keys())
similarity_dn_up = pd.DataFrame(index=diseses_genes.keys(), columns=diseses_genes.keys())

In [17]:
#calculating and recording Jaccard index between genes of up-regulated into the dataframe

for d1 in similarity_up_up.index:
    for d2 in similarity_up_up.columns:
        gs1 = set(diseses_genes[d1]['up'])
        gs2 = set(diseses_genes[d2]['up'])
        similarity_up_up.loc[d1, d2] = calculate_jacard(gs1, gs2)
        

In [21]:
similarity_up_up

,dz:328,dz:325,dz:324,dz:323,dz:322,dz:321,dz:320,dz:49,dz:822,dz:823,...,dz:658,dz:653,dz:849,dz:651,dz:650,dz:657,dz:656,dz:655,dz:654,dz:439
dz:328,1,0.0336283,0.0425139,0.0189655,0.0444965,0.0829384,0.0700809,0,0.0253456,0.022113,...,0.0348584,0.0584416,0.0371134,0.0982906,0.103594,0.0521236,0.0851927,0.0593081,0.107623,0.00597015
dz:325,0.0336283,1,0.092511,0.0198413,0.0592217,0.0907534,0.0230496,0,0.012966,0.0170648,...,0.25239,0.0612245,0.0945513,0.0499244,0.0556391,0.077266,0.158833,0.106183,0.10311,0.00588928
dz:324,0.0425139,0.092511,1,0.0301783,0.032368,0.0656304,0.0220183,0,0.0202361,0.024911,...,0.0986159,0.0740741,0.0710824,0.0433437,0.0428135,0.0829493,0.0962145,0.0764075,0.10101,0.00481928
dz:323,0.0189655,0.0198413,0.0301783,1,0.0209677,0.0238474,0.0245614,0,0.0446281,0.0100503,...,0.0137825,0.0281863,0.0192024,0.0130058,0.0143062,0.0280899,0.02849,0.0221675,0.0225225,0.00819672
dz:322,0.0444965,0.0592217,0.032368,0.0209677,1,0.0549683,0.02331,0,0.0103734,0.0132743,...,0.055102,0.0499244,0.032197,0.047081,0.0502793,0.0463458,0.0785047,0.0570988,0.0677291,0.0156028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dz:657,0.0521236,0.077266,0.0829493,0.0280899,0.0463458,0.04363,0.0247619,0,0.0173611,0.0469925,...,0.128205,0.0545213,0.133803,0.0702614,0.0591054,1,0.136134,0.0739726,0.117958,0.0162095
dz:656,0.0851927,0.158833,0.0962145,0.02849,0.0785047,0.105263,0.0434783,0,0.0123023,0.0281955,...,0.163148,0.08,0.0874786,0.0858586,0.0847176,0.136134,1,0.120116,0.126126,0.0138539
dz:655,0.0593081,0.106183,0.0764075,0.0221675,0.0570988,0.0466165,0.0512397,0,0.0506912,0.0685155,...,0.11215,0.0696279,0.06,0.110619,0.110949,0.0739726,0.120116,1,0.0763583,0.00995575
dz:654,0.107623,0.10311,0.10101,0.0225225,0.0677291,0.118609,0.0317797,0,0.00753296,0.0201613,...,0.121032,0.066092,0.144788,0.0633803,0.0643478,0.117958,0.126126,0.0763583,1,0.00791557


In [18]:
#calculating and recording Jaccard index between genes of down-regulated into the dataframe

for d1 in similarity_dn_dn.index:
    for d2 in similarity_dn_dn.columns:
        gs1 = set(diseses_genes[d1]['dn'])
        gs2 = set(diseses_genes[d2]['dn'])
        similarity_dn_dn.loc[d1, d2] = calculate_jacard(gs1, gs2)

In [22]:
similarity_dn_dn

,dz:328,dz:325,dz:324,dz:323,dz:322,dz:321,dz:320,dz:49,dz:822,dz:823,...,dz:658,dz:653,dz:849,dz:651,dz:650,dz:657,dz:656,dz:655,dz:654,dz:439
dz:328,1,0.0234899,0.0177994,0.0116861,0.0431755,0.0409605,0.136558,0,0.0205761,0.0793872,...,0.0169014,0.00383877,0.0267062,0.0848,0.0841424,0.0483871,0.0217054,0.00925926,0.0525526,0.00191205
dz:325,0.0234899,1,0.037123,0.00236407,0.0253165,0.0296846,0.0215232,0,0.0181159,0.0136752,...,0.0325048,0.00589971,0.0240964,0.0101833,0.0124481,0.030837,0.0148936,0.00276243,0.0400802,0
dz:324,0.0177994,0.037123,1,0.00681818,0.0353357,0.0891841,0.0324675,0,0.0338078,0.0372881,...,0.0429104,0.01983,0.0752033,0.0341365,0.0325866,0.0822222,0.0420168,0.0186667,0.082495,0.00837989
dz:323,0.0116861,0.00236407,0.00681818,1,0.0199275,0.014733,0.00491803,0,0.0127042,0.0155172,...,0.0094162,0.0150602,0.00596421,0.00819672,0.00623701,0.0153173,0.0172043,0,0.00194553,0
dz:322,0.0431755,0.0253165,0.0353357,0.0199275,1,0.0644172,0.05,0,0.0494012,0.042735,...,0.0210526,0.0147992,0.0709571,0.0258481,0.0245098,0.0976492,0.0283806,0.0100604,0.0647249,0.0169133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dz:657,0.0483871,0.030837,0.0822222,0.0153173,0.0976492,0.0682226,0.0445151,0,0.0524476,0.0656566,...,0.0622711,0.016,0.136364,0.0268199,0.0252427,1,0.055102,0.0100251,0.115768,0.00526316
dz:656,0.0217054,0.0148936,0.0420168,0.0172043,0.0283806,0.070922,0.0261941,0,0.00825083,0.0206677,...,0.040636,0.0290237,0.0351852,0.0244361,0.0209125,0.055102,1,0.0248756,0.0839695,0.0025641
dz:655,0.00925926,0.00276243,0.0186667,0,0.0100604,0.00409836,0.0222222,0,0.033264,0.0292398,...,0.0106383,0,0.00907029,0.021327,0.0192771,0.0100251,0.0248756,1,0.0202247,0.00362319
dz:654,0.0525526,0.0400802,0.082495,0.00194553,0.0647249,0.0986395,0.0504451,0,0.0219092,0.0410959,...,0.0786325,0.0536585,0.155769,0.0262238,0.0284192,0.115768,0.0839695,0.0202247,1,0


In [19]:
#calculating and recording Jaccard index between genes of up-regulated and down-regulated respictevly 

for d1 in similarity_up_dn.index:
    for d2 in similarity_up_dn.columns:
        gs1 = set(diseses_genes[d1]['up'])
        gs2 = set(diseses_genes[d2]['dn'])
        similarity_up_dn.loc[d1, d2] = calculate_jacard(gs1, gs2)

In [23]:
similarity_up_dn

,dz:328,dz:325,dz:324,dz:323,dz:322,dz:321,dz:320,dz:49,dz:822,dz:823,...,dz:658,dz:653,dz:849,dz:651,dz:650,dz:657,dz:656,dz:655,dz:654,dz:439
dz:328,0,0.0146341,0.0308057,0.0223325,0.0632184,0.0303605,0.0558464,0,0.0763209,0.0699816,...,0.0373281,0.0123077,0.0462185,0.0364026,0.0347826,0.0555556,0.0333333,0.0203488,0.0304878,0.0280374
dz:325,0.0499325,0,0.0865724,0,0.0921248,0.112308,0.0782967,0,0.0503597,0.0410397,...,0.0396476,0.00992063,0.14527,0.0310559,0.0330709,0.104167,0.0238095,0.0172414,0.128079,0.00196464
dz:324,0.0397805,0.0341113,0,0.0141844,0.113707,0.117647,0.0379919,0,0.0456554,0.0407303,...,0.0650155,0.0208768,0.0786885,0.0222222,0.0192308,0.0731707,0.059322,0.028169,0.0952381,0.0165975
dz:323,0.0651289,0.0083612,0.0196721,0,0.0108992,0.035461,0.03125,0,0.00820793,0.0253672,...,0.0468521,0.0078125,0.0285285,0.0228659,0.0231481,0.0255183,0.0219436,0.00749064,0.0373692,0.00193798
dz:322,0.0223642,0.0155211,0.06,0.0111359,0,0.0483871,0.0302548,0,0.0331588,0.0366057,...,0.0344828,0.0136612,0.0485437,0.0193798,0.0216963,0.0550847,0.0263158,0.00511509,0.0397727,0.00540541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dz:657,0.0364656,0.0529301,0.0627306,0.00362976,0.045045,0.0926518,0.0492264,0,0.058193,0.0255682,...,0.0502355,0.0107527,0.0191388,0.0179153,0.0232172,0,0.032368,0.014433,0.0401284,0.00856531
dz:656,0.0474138,0.049904,0.0863724,0.00929368,0.101124,0.0995106,0.0697674,0,0.0673981,0.064275,...,0.066343,0.0132159,0.0882353,0.0318792,0.0323129,0.131021,0,0.0147368,0.0868825,0.00655022
dz:655,0.0941176,0.0380349,0.053125,0.00773994,0.0802721,0.0567568,0.0890323,0,0.140173,0.137309,...,0.0295302,0.0106762,0.0604317,0.0493469,0.0437956,0.0643185,0.0217707,0,0.058156,0.00886525
dz:654,0.0253353,0.0563674,0.0824742,0.0286885,0.075,0.132379,0.0342262,0,0.0613599,0.0484375,...,0.163842,0.00721154,0.0613718,0.0195382,0.0198198,0.0581395,0.0315985,0.00455581,0,0.0218978


In [20]:
#calculating and recording Jaccard index between genes of down-regulated and up-regulated respictevly 

for d1 in similarity_dn_up.index:
    for d2 in similarity_dn_up.columns:
        gs1 = set(diseses_genes[d1]['dn'])
        gs2 = set(diseses_genes[d2]['up'])
        similarity_dn_up.loc[d1, d2] = calculate_jacard(gs1, gs2)

In [24]:
similarity_dn_up

,dz:328,dz:325,dz:324,dz:323,dz:322,dz:321,dz:320,dz:49,dz:822,dz:823,...,dz:658,dz:653,dz:849,dz:651,dz:650,dz:657,dz:656,dz:655,dz:654,dz:439
dz:328,0,0.0499325,0.0397805,0.0651289,0.0223642,0.00773994,0.0572451,0,0.155515,0.0252101,...,0.0535433,0.0354957,0.0265096,0.0727273,0.0799397,0.0364656,0.0474138,0.0941176,0.0253353,0.0152047
dz:325,0.0146341,0,0.0341113,0.0083612,0.0155211,0.0240175,0.0123762,0,0.0133038,0.0118203,...,0.0822222,0.0215385,0.114719,0.0395257,0.0289017,0.0529301,0.049904,0.0380349,0.0563674,0.0254111
dz:324,0.0308057,0.0865724,0,0.0196721,0.06,0.0868597,0.0214797,0,0.017094,0.0347222,...,0.112088,0.0459418,0.0744467,0.0380952,0.0433962,0.0627306,0.0863724,0.053125,0.0824742,0.00858369
dz:323,0.0223325,0,0.0141844,0,0.0111359,0.0152838,0.00746269,0,0.0202703,0.00952381,...,0,0.00609756,0.00392927,0.00772201,0.00760456,0.00362976,0.00929368,0.00773994,0.0286885,0.0118694
dz:322,0.0632184,0.0921248,0.113707,0.0108992,0,0.0537262,0.0300752,0,0.0101695,0.0143113,...,0.0886957,0.0621693,0.0315457,0.0488959,0.0466563,0.045045,0.101124,0.0802721,0.075,0.00732601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dz:657,0.0555556,0.104167,0.0731707,0.0255183,0.0550847,0.106522,0.0251142,0,0.0268595,0.0173913,...,0.0979167,0.0764526,0.0296846,0.0442804,0.053211,0,0.131021,0.0643185,0.0581395,0.00414938
dz:656,0.0333333,0.0238095,0.059322,0.0219436,0.0263158,0.0277778,0.0315315,0,0.0284553,0.0127389,...,0.0229008,0.02149,0.0405904,0.0176991,0.0156794,0.032368,0,0.0217707,0.0315985,0.00272851
dz:655,0.0203488,0.0172414,0.028169,0.00749064,0.00511509,0.0176322,0.0177515,0,0.0103093,0.0371429,...,0.0168675,0.0239316,0.0135135,0.0244444,0.0240175,0.014433,0.0147368,0,0.00455581,0.00323102
dz:654,0.0304878,0.128079,0.0952381,0.0373692,0.0397727,0.0980392,0.026694,0,0.0281426,0.0136719,...,0.128906,0.0678925,0.061296,0.0317726,0.0330579,0.0401284,0.0868825,0.058156,0,0.00647668


In [25]:
#creating a general similarity table

general_similiraty = similarity_up_up + similarity_dn_dn - similarity_dn_up - similarity_up_dn 

In [26]:
general_similiraty.head()

,dz:328,dz:325,dz:324,dz:323,dz:322,dz:321,dz:320,dz:49,dz:822,dz:823,...,dz:658,dz:653,dz:849,dz:651,dz:650,dz:657,dz:656,dz:655,dz:654,dz:439
dz:328,2,-0.00744842,-0.010273,-0.0568097,0.00208937,0.0857984,0.093547,0,-0.185915,0.00630854,...,-0.0391116,0.0144769,-0.00890843,0.0739608,0.0730142,0.00848946,0.026151,-0.0458992,0.104353,-0.0353599
dz:325,-0.00744842,2,0.00895023,0.0138441,-0.0231078,-0.0158871,-0.0461001,0,-0.0325816,-0.0221199,...,0.163025,0.0356651,-0.141341,-0.0104739,0.00611463,-0.0489937,0.100013,0.053669,-0.0412564,-0.0214864
dz:324,-0.010273,0.00895023,2,0.00313998,-0.106004,-0.0496923,-0.00498569,0,-0.00870548,-0.0132534,...,-0.035577,0.0270855,-0.00684956,0.0171627,0.012773,0.0292702,-0.00746308,0.0137802,0.00579275,-0.011982
dz:323,-0.0568097,0.0138441,0.00313998,2,0.0188602,-0.0121645,-0.00923325,0,0.0288541,-0.00932347,...,-0.0236534,0.0293365,-0.00729122,-0.00938536,-0.0102096,0.0142591,0.0144571,0.00693691,-0.0415897,-0.0056107
dz:322,0.00208937,-0.0231078,-0.106004,0.0188602,2,0.0172722,0.0129801,0,0.0164463,0.00509245,...,-0.0470237,-0.011107,0.0230646,0.00465338,0.00643658,0.0438652,-0.0205541,-0.0182281,0.0176813,0.0197847


In [44]:
similarity_up_up.to_csv('../results/similarity_up_up.csv')
similarity_dn_dn.to_csv('../results/similarity_dn_dn.csv')
similarity_up_dn.to_csv('../results/similarity_up_dn.csv')
similarity_dn_up.to_csv('../results/similarity_dn_up.csv')
general_similiraty.to_csv('../results/general_similiraty.csv')